# Linear Bias correction from Netcdf file.
### Requirement
1. Netcdf file of observed data in monthly scale
2. Netcdf file of raw data in daily scale

In [1]:
import pandas as pd
import numpy as np
import xarray as xr

# Extracting Values from Netcdf

#Openning files station file
station = pd.read_csv(r'\stations.csv')

# Opening Netcdf files observation and raw netcdf files
ds_obs = xr.open_dataset(r'\Obs.nc')
ds_raw = xr.open_dataset(r'\raw.nc')


obs = []
raw = []
sta = []
for index,row in station.iterrows():
    station_name='PStn'+str(int(row['Point']))
    
    lon=float(row['lon'])
    lat=float(row['lat'])
    x,y=(lon,lat)
    
    o = ds_obs.PPT.sel(lon =x, lat = y, method ='nearest' ).values # Change for the temperature variable name
    a = ds_raw.precip.sel(lon =x, lat = y, method ='nearest' ).values # Change for the temperature variable name
    
    obs.append(o)
    raw.append(a)
    sta.append(station_name)
    
obs = pd.DataFrame(np.transpose(obs), columns =[sta])
raw = pd.DataFrame(np.transpose(raw), columns =[sta])

#---------------------------------------------------------------------------------------------------------------------------
# Bias correction

# setting time range. In this example we assumed starting data from (1991 to 2010)
time_obs = pd.date_range(start = '1991-01-01', end = '2010-12-31', freq = 'm') # observation time is in monthly formate
time_raw = pd.date_range(start = '1991-01-01', end = '2010-12-31', freq = 'd') # raw data is in daily formate

# setting data frames
bias_corr = pd.DataFrame(dict(time = time_raw))
factor = []


for st in sta:
    
    # station name
    sta_name = str(st)


    #observed monthly data 
    st_obs = pd.DataFrame(dict(time = time_obs))
    st_obs['obs'] = obs[str(st)]

    #raw daily data
    st_raw = pd.DataFrame(dict(time = time_raw))
    st_raw['raw'] = raw[str(st)]
    
    #taking 12 monthly mean of both raw and observation data
    obsi = st_obs.groupby(st_obs.time.dt.month).mean()
    rawi = st_raw.set_index('time').resample('M').sum().reset_index().groupby(st_raw.time.dt.month).mean()

    #shifting data in one single dataframe
    obsi['raw'] = rawi
    
    #calculating factors
    obsi['factor'] = obsi.obs/obsi.raw
    
    #appeding factors in single list
    factor.append(obsi.factor)
    
    #multipling factor to raw data,rescaling data to daily scale and saving it in bias_corr dataframe.
    bias_corr[str(st)] = st_raw['raw'].mul(obsi.factor.reindex(st_raw.time.dt.month).values, axis=0)
    


In [15]:
st_raw.set_index('time').resample('M').sum().reset_index().groupby(st_raw.time.dt.month).mean()

,raw
time,
1,687.437439
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000


## Save result to excel file

In [2]:
bias_corr.to_excel(r'C:\Users\Sibtain Shah\Desktop\test\netcdf_project\lr_precipitation.xlsx')
factor.to_excel(r'C:\Users\Sibtain Shah\Desktop\test\netcdf_project\lr_factor_precipitation.xlsx')

,time,PStn1,PStn2,PStn3,PStn4,PStn5,PStn6,PStn7,PStn8,PStn9,...,PStn416,PStn417,PStn418,PStn419,PStn420,PStn421,PStn422,PStn423,PStn424,PStn425
0,1991-01-31,4.141396,4.168743,4.118755,2.975196,4.128444,4.158621,4.112251,3.194220,3.257846,...,32.661072,32.818455,32.916870,32.322060,66.742790,73.359322,72.646187,68.905449,76.888977,72.395515
1,1991-02-28,48.551205,48.782738,49.080879,50.319294,48.579506,48.826492,49.139519,56.951157,55.241085,...,124.556870,150.963455,150.329773,159.388535,298.112518,326.748871,322.370941,283.906860,323.377258,291.498260
2,1991-03-31,35.574146,35.789890,35.660328,30.340778,35.637505,35.870529,35.753735,32.993946,32.708359,...,124.489647,121.051193,121.660118,116.675560,191.459152,211.093765,210.118271,196.492416,214.396881,202.261200
3,1991-04-30,101.758133,101.782944,102.377731,111.049789,101.566200,101.612122,102.234474,104.457092,105.406364,...,123.195633,128.024750,130.146332,140.425095,191.157349,207.825882,203.725388,187.325638,192.739960,184.371033
4,1991-05-31,22.014458,22.195284,22.308899,21.616207,22.049389,22.242214,22.365480,22.876926,23.204802,...,57.237995,59.365593,59.774837,53.204407,72.419853,79.877884,78.351936,71.201332,79.103836,74.235817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2010-08-31,297.459442,304.192535,307.039154,250.516022,294.598480,302.690643,306.885803,271.608612,295.313629,...,281.686249,271.117737,272.654175,261.014618,275.480957,256.560059,251.898758,256.664520,247.124695,256.273438
236,2010-09-30,97.818420,99.607178,101.430206,104.328056,97.721611,99.835571,101.950836,110.473381,110.211639,...,39.418797,49.774559,50.419495,62.862385,81.432510,79.629021,77.262184,63.072479,78.899277,63.609138
237,2010-10-31,15.334452,15.340257,15.505353,17.702864,15.252348,15.276904,15.460950,18.043594,19.171877,...,12.331779,18.138643,17.969500,22.328724,38.257034,41.671154,40.934452,41.903027,38.787952,42.281769
238,2010-11-30,6.833189,6.804724,6.806985,7.227473,6.867316,6.818767,6.802400,8.223395,7.978643,...,1.365193,1.562065,1.586072,1.874238,3.477987,4.205261,4.154213,3.956692,3.919966,3.922995


# Temperature Bias Correction

In [31]:
import pandas as pd
import numpy as np
import xarray as xr

# Extracting Values from Netcdf

#Openning files station file
station = pd.read_csv(r'\stations.csv')

# Opening Netcdf files observation and raw netcdf files
ds_obs = xr.open_dataset(r'\Obs.nc')
ds_raw = xr.open_dataset(r'\Raw.nc')


obs = []
raw = []
sta = []
for index,row in station.iterrows():
    station_name='PStn'+str(int(row['Point']))
    
    lon=float(row['lon'])
    lat=float(row['lat'])
    x,y=(lon,lat)
    
    ## Insert the name of temperature variables
    o = ds_obs.PPT.sel(lon =x, lat = y, method ='nearest' ).values # Change for the temperature variable name
    a = ds_raw.precip.sel(lon =x, lat = y, method ='nearest' ).values # Change for the temperature variable name
    
    obs.append(o)
    raw.append(a)
    sta.append(station_name)
    
obs = pd.DataFrame(np.transpose(obs), columns =[sta])
raw = pd.DataFrame(np.transpose(raw), columns =[sta])

#---------------------------------------------------------------------------------------------------------------------------
# Bias correction

# setting time range
time_obs = pd.date_range(start = '1991-01-01', end = '2010-12-31', freq = 'm') # set date range
time_raw = pd.date_range(start = '1991-01-01', end = '2010-12-31', freq = 'd') # for daily data correct the "freq to 'd'"

# setting data frames
bias_corr = pd.DataFrame(dict(time = time_raw))
factor = []


for st in sta:
    
    # station name
    sta_name = str(st)


    #observed monthly data 
    st_obs = pd.DataFrame(dict(time = time_obs))
    st_obs['obs'] = obs[str(st)]

    #raw daily data
    st_raw = pd.DataFrame(dict(time = time_raw))
    st_raw['raw'] = raw[str(st)]
    
    #taking 12 monthly mean of both raw and observation data
    obsi = st_obs.groupby(st_obs.time.dt.month).mean()
    rawi = st_raw.groupby(st_raw.time.dt.month).mean()

    #shifting data in one single dataframe
    obsi['raw'] = rawi
    
    #calculating factors
    obsi['factor'] = obsi.obs - obsi.raw
    
    #appeding factors in single list
    factor.append(obsi.factor)
    
    #multipling factor to raw data,rescaling data to daily scale and saving it in bias_corr dataframe.
    bias_corr[str(st)] = np.array(st_raw['raw']) + np.array(obsi.factor.reindex(st_raw.time.dt.month))
    


In [ ]:
bias_corr.to_excel(r'C:\Users\Sibtain Shah\Desktop\test\netcdf_project\lr_temp.xlsx')
factor.to_excel(r'C:\Users\Sibtain Shah\Desktop\test\netcdf_project\lr_factor_temp.xlsx')